In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:
BNP_diff = pd.read_excel('BNP_diff.xlsx', sheet_name='Ekonomifakta')

Unemp = pd.read_excel('Unemp.xlsx', sheet_name='Ekonomifakta')

<h3>Here we will try to join the unemployemnt and BNP growth data to check for correlation. Note that the BNP growth data is quarterly and the unemployment data is monthly</h3>

<h3>First we fix the dates</h3>

In [ ]:
Unemp.dropna(axis=0, inplace=True)

In [ ]:
def month_recode(month):
    if month == "maj":
        return 'may'
    if month == 'okt':
        return 'oct'
    else:
        return month

In [ ]:
Unemp['Month_2']=Unemp['Month'].str.slice(0,3).apply(month_recode)

In [ ]:
Unemp['Month_3']=Unemp['Month_2'].str.cat(Unemp['Month'].str[-2:],sep='-').astype('string')

In [ ]:
import datetime as dt



Unemp['Date']=pd.to_datetime(Unemp['Month_3'], format='%b-%y')+ pd.offsets.MonthEnd()

In [ ]:
Unemp.head()

In [ ]:
Unemp_2=Unemp.drop(columns=['Month','Procent, trend','Procent, säsongsjusterat','Month_2','Month_3'])

In [ ]:
Unemp_3=Unemp_2.rename(columns={'Procent': 'Unemp'})

In [ ]:
BNP_diff.dropna(axis=0, inplace=True)

In [ ]:
def quarter_recode(quarter):
    if quarter == ", kv 1":
        return '03-31'
    if quarter == ', kv 2':
        return '06-30'
    if quarter == ', kv 3':
        return '09-30'
    if quarter == ', kv 4':
        return '12-31'
    else:
        return 'Error' 

In [ ]:
BNP_diff['Year']=BNP_diff['Quarter'].str.slice(0,4)

In [ ]:
BNP_diff['Date']=pd.to_datetime(BNP_diff['Year'].str.cat(BNP_diff['Quarter'].str[-6:].apply(quarter_recode),sep='-').astype('string'),format='%Y-%m-%d')

In [ ]:
BNP_diff_2=BNP_diff.rename(columns={'Procent, jämfört med föregående kvartal': 'BNP growth'})

In [ ]:
BNP_diff_3=BNP_diff_2.drop(columns=['Quarter','Procent, jämfört med samma kvartal föregående år'])

<h3>Now let's join</h3>

In [ ]:
BNP_unemp=pd.merge(BNP_diff_3, Unemp_3, on='Date')

In [ ]:
BNP_unemp.head(20)


<h3>Let's check the correlation between BNP growth and Unemp</h3>

In [ ]:
BNP_unemp['Unemp'].corr(BNP_unemp['BNP growth'])

<h3>Standardise the data and plot display series in one plot</h3>

In [ ]:
from scipy.stats import zscore

BNP_unemp['Unemp z'] = zscore(BNP_unemp['Unemp'])
BNP_unemp['BNP growth z']=zscore(BNP_unemp['BNP growth'])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(BNP_unemp['Date'],BNP_unemp['Unemp z'],'r', label='Unemplyment z-score')
plt.plot(BNP_unemp['Date'],BNP_unemp['BNP growth z'],'g', label='BNP growth z-score')
plt.xlabel('Year')
plt.ylabel('z-score')
plt.legend(loc='lower left')

plt.show()

In [ ]:
BNP_unemp_m=Unemp_3.merge(BNP_diff_3, how='left', on='Date')

In [ ]:
BNP_unemp_m.head(20)

In [ ]:
BNP_unemp_m['BNP growth interp']=pd.Series(BNP_unemp_m['BNP growth']).interpolate().tolist()

In [ ]:
BNP_unemp_m.head(20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(BNP_unemp_m['Date'],BNP_unemp_m['Unemp'],'r', label='Unemployment')
plt.plot(BNP_unemp_m['Date'],BNP_unemp_m['BNP growth interp'],'g', label='BNP growth')
plt.xlabel('Year')
plt.ylabel('Percent')
plt.legend(loc='lower left')

plt.show()